# Imports

In [2]:
from erpack.data_cleaning import *
from erpackage.feature_engineering import *
from erpackage.feature_selection import *
from erpackage.modeling import *
from erpackage.error_analysis import *
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

ImportError: cannot import name 'train_test_split' from 'erpack.train_test_split' (C:\Users\Gipsy\Documents\ericpackage\erpack\train_test_split\__init__.py)

# Leitura de dados

In [ ]:
caminho = 'C://Users//mxcy//Documents//'
dados = pd.read_csv(caminho + 'houseprice.csv', encoding='utf8')

# Sanity Check

In [ ]:
#profile = ProfileReport(dados, title='Pandas Profiling Report', explorative=True)

In [ ]:
#profile.to_file("your_report.html")

In [ ]:
dados.head()

In [ ]:
tabela, lista_cat, lista_num = dc_features_tipos(dados)

In [ ]:
lista_num

In [ ]:
lista_cat

In [ ]:
tabela, lista_cat_nulos, lista_num_nulos = dc_nulos(dados)

In [ ]:
tabela.head()

In [ ]:
lista_num_nulos

# Separação de conjunto de dados

In [ ]:
target= 'SalePrice'
X = dados.iloc[:, dados.columns != target]
y = dados.loc[:, target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=True)

# Limpeza de dados

###### Drop colunas:

In [ ]:
lista_colunas_retirar = ['Id','PoolQC', 'SalePrice']

In [ ]:
def retira_colunas(dataset,lista_colunas_retirar):
    print('qtd colunas antes: '+str(dataset.shape[1]))
    dataset.drop(columns = lista_colunas_retirar, inplace=True, errors = 'ignore')
    print('qtd colunas depois: '+str(dataset.shape[1]))
    return dataset

dados = retira_colunas(X_train,lista_colunas_retirar)

###### Atualização das listas:

In [ ]:
lista_num_nulos =[i for i in lista_num_nulos if i not in lista_colunas_retirar] 
lista_cat_nulos = [i for i in lista_cat_nulos if i not in lista_colunas_retirar]
lista_num = [i for i in lista_num if i not in lista_colunas_retirar]
lista_cat = [i for i in lista_cat if i not in lista_colunas_retirar]

##### Drop de registros (duplicatas e outliers):

In [ ]:
X_train, y_train = dc_duplicatas(X_train, y_train)

##### Impute para missing data:

In [ ]:
lista_variaveis = lista_num_nulos 
forma = 'mean'

In [ ]:
X_train, imputer1 = dc_fillna_custom(X_train, lista_variaveis, forma)
#X_val = imputer1.transform(X_val)

In [ ]:
lista_variaveis = lista_cat_nulos 
forma = 'categorical'
X_train, imputer2 = dc_fillna_custom(X_train, lista_variaveis, forma)
#X_val = imputer1.transform(X_val)

# Feature Engineering

##### Transformações Numéricas

In [ ]:
lista_num = lista_num

In [ ]:
print(lista_variaveis)

In [ ]:
lista_variaveis = lista_num
forma = 'Log' 
X_train2, tf1, lista_variaveis_trans1 = fe_numerical_transform(X_train, lista_variaveis, forma, valor=None)
print(len(X_train2.columns))
X_train = original_e_transformada(X_train,X_train2, lista_variaveis_trans1, sufixo = forma)
X_train.head(1)

In [ ]:
forma='Reciprocal'
X_train2, tf2, lista_variaveis_trans2 = fe_numerical_transform(X_train, lista_variaveis, forma, valor=None)
print(len(X_train2.columns))
X_train = original_e_transformada(X_train,X_train2, lista_variaveis_trans2, sufixo = forma)
X_train.head(1)

In [ ]:
forma='Power'
X_train2, tf3, lista_variaveis_trans3 = fe_numerical_transform(X_train, lista_variaveis, forma, valor=1.5)
print(len(X_train2.columns))
X_train = original_e_transformada(X_train,X_train2, lista_variaveis_trans3, sufixo = forma)
X_train.head(1)

forma='BoxCox'
X_train2, tf4, lista_variaveis_trans4 = fe_numerical_transform(X_train, lista_variaveis, forma, valor=None)
print(len(X_train2.columns))
X_train = original_e_transformada(X_train,X_train2, lista_variaveis_trans4, sufixo = forma)
X_train.head(1)

forma='YeoJohnson'
X_train2, tf5, lista_variaveis_trans5 = fe_numerical_transform(X_train, lista_variaveis, forma, valor=None)
print(len(X_train2.columns))
X_train = original_e_transformada(X_train,X_train2, lista_variaveis_trans5, sufixo = forma)
X_train.head(1)

##### Transformações categóricas

In [ ]:
lista_variaveis = lista_cat
forma = 'ohe'

X_train, encoder1 = fe_categorical_transform(X_train, y_train, lista_variaveis, forma , valor=None)

In [ ]:
forma = 'RareLabel'

In [ ]:
X_train, encoder1 = fe_categorical_transform(X_train, y_train, lista_variaveis, forma , valor=None)

In [ ]:
forma = 'CountFrequency'

In [ ]:
X_train, encoder2 = fe_categorical_transform(X_train, y_train, lista_variaveis, forma , valor=None)

In [ ]:
X_train.head(2)

In [ ]:
len(X_train.columns)

# Feature Selection

In [ ]:
X_train, lista_colunas_selecionadas = fs_variancia(X_train,  0.01, ver_features_selecionadas=True)

In [ ]:
set(lista_colunas_selecionadas)

# Modeling

##### Preparação do conjunto de Validação

In [ ]:
X_val = retira_colunas(X_val, lista_colunas_retirar)

In [ ]:
X_val = imputer1.transform(X_val)
X_val = imputer2.transform(X_val)

In [ ]:
X_val2 = tf1.transform(X_val) 
X_val = original_e_transformada(X_val,X_val2, lista_variaveis_trans1, sufixo = 'Log')
X_val2 = tf2.transform(X_val) 
X_val = original_e_transformada(X_val,X_val2, lista_variaveis_trans2, sufixo = 'Reciprocal')
X_val2 = tf3.transform(X_val) 
X_val = original_e_transformada(X_val,X_val2, lista_variaveis_trans3, sufixo = 'Power')
#X_val2 = tf4.transform(X_val) 
#X_val = original_e_transformada(X_val,X_val2, lista_variaveis_trans4, sufixo = 'BoxCox')
#X_val2 = tf5.transform(X_val) 
#X_val = original_e_transformada(X_val,X_val2, lista_variaveis_trans5, sufixo = 'YeoJohnson')

In [ ]:
X_val = encoder1.transform(X_val)
X_val = encoder2.transform(X_val)


In [ ]:
encoder1.encoder_dict_

In [ ]:
X_val = X_val[lista_colunas_selecionadas]

In [ ]:
tabela, lista_cat_nulos, lista_num_nulos = dc_nulos(X_val)
lista_num_nulos

In [ ]:
modelos, lista_predicoes = auto_ml_regressor(
    X_train,
    X_val,
    y_train,
    y_val,
    algoritmos=['randomforest', 'extratree', 'gbm', 'adaboost', 'xgboost'],
    search=True,
    type_search='randomsearch',
    target_transform=None)
modelos

# Avaliação de resultados

In [ ]:
modelos.iloc[0,1]

In [ ]:
decomposicao_bias_variancia(modelos.iloc[0,1], X_train, y_train, X_val, y_val)

In [ ]:
report_features_erros_modelos(modelos, X_val, y_val, lista_predicoes, target)

# Predição e interpretação